### Installs and imports

In [ ]:
#!pip3 install --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint
#!python3 -m pip install twint
!pip install twint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import twint
import pandas as pd

### Load the dataset and extract usernames

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the CSV

In [ ]:
#df = pd.read_csv ('/content/drive/MyDrive/replies-larrouturou-annotated-botometer.csv')
df = pd.read_csv ('/content/drive/MyDrive/replies-alviinaalametsa-annotated-botometer.csv')

In [ ]:
df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'sentiment', 'annotator', 'annotation_id', 'updated_at',
       'lead_time', 'botometerCapEng.score', 'botometerCapUni.score',
       'botometerOverallRawEng.score', 'botometerOverallRawUni.score',
       'botometerDisplayEng.score', 'botometerDisplayUni.score'],
      dtype='object')

#### Extract usernames of potentially bot accounts

In [ ]:
df_bot = df[df['botometerOverallRawUni.score'] > 0.6]

Drop duplicates

In [ ]:
df_bot_unique = df_bot.drop_duplicates(subset=['username'])

Convert to a list

In [ ]:
username_list_bot = df_bot_unique ['username'].tolist()
print(len(username_list_bot))
print (username_list_bot)

8
['vt82878', 'sanjuktaroyvai1', 'ambar_dave', 'nameshakehe', 'SanatanForce', 'vedalasrinivas2', 'kirudevadiga', 'PunctureWalIa']


#### Extract usernames of 100% authentic/ real accounts

In [ ]:
df_real = df[df['botometerOverallRawUni.score'] < 0.1]

Drop duplicates

In [ ]:
df_real_unique = df_real.drop_duplicates(subset=['username'])

Convert to a list

In [ ]:
username_list_real = df_real_unique ['username'].tolist()
print(len(username_list_real))
print (username_list_real)

35
['tarunsharma005', 'ShekharrG', 'Sunishen', 'khab33b', 'IndiaFirst2022', 'bhanuvarma007', 'Ksharkhowa', 'tarunlalan', 'roshovani', 'pairamblr', 'tellitaly', 'spandakarika108', 'MohanKumarL11', 'Nishwins1', 'abhiawakes', 'niranjan_shaha', 'dcemeterygirl', 'UnApologeticM1', 'citizenNA', 'Prasenjit97m', 'AkashD60789383', 'Arjun_S_R', 'dschamyal', 'ScorpionHere', 'PrasadSatya10', 'VickyHanumant', 'woke_less', 'ib4uanytime', 'SachinDPatange', 'DiVpops', 'MiFe007', 'bhattketan1468', '_hamza_iftikhar', 'Hilale_pakistan', 'DrBharatbhushan']


### Data acquisition function

Check out [Twint's Github page](https://github.com/twintproject/twint) for more details.



> See the [configuration](https://github.com/twintproject/twint/wiki/Configuration) page at Twint wiki page for more info on the the parameters to be set in specific scraping functions.

In [ ]:
def data_acq_search (user_name, num, since_date, until_date):
    print ("======================================")
    print(":: Acquiring Data for", user_name, "::")
    print ("======================================")

    # Configure
    c = twint.Config()

    #c.Search = search_string    # uncomment this if you want to scrape using a search_string, and provide this as the input of this function
    c.Username = user_name

    c.Since = since_date
    c.Until = until_date

    c.Limit = num

    #c.Lang = 'nl'
    #c.Translate = True 
    #c.TranslateDest = "en"

    c.Store_object =  True
    c.User_full = True
    c.Profile_full = True
    c.Show_hashtags = True
    c.Hide_output = True

    c.Pandas = True
    twint.run.Search(c)

    return twint.storage.panda.Tweets_df

In [ ]:
# ===== redundant function =====
# ===== the call breaks due to JSON error on the Twint end ==== 

def data_acq_timeline (user_name, num, since_date, until_date):
    print ("======================================")
    print(":: Acquiring Data for", user_name, "::")
    print ("======================================")

    # Configure
    c = twint.Config()

    #c.Search = search_string    # uncomment this if you want to scrape using a search_string, and provide this as the input of this function
    c.Username = user_name

    c.Since = since_date
    c.Until = until_date

    c.Limit = num

    #c.Lang = 'nl'
    #c.Translate = True 
    #c.TranslateDest = "en"

    c.Store_object =  True
    c.User_full = True
    c.Profile_full = True
    c.Hide_output = True

    c.Pandas = True
    twint.run.Profile(c)

    return twint.storage.panda.Tweets_df

### Parameters to acquire tweets

In [ ]:
# provide the list of usernames

#----larrouturou-----
#username_list = username_list_bot
#username_list = list (set(username_list_real) - set(['mrImteyaz8','TheWhiteWaIker','frankblunt2021', 'maverick9762', 'dinakar24', 'Prasenjit97m', 'omprakash2711']))


#----alviinaalametsa----
#username_list = username_list_bot
username_list = list (set(username_list_real) - set(['spandakarika108', 'Prasenjit97m']))


# provide the number of most recent tweets you want to scrape
N = 10000

# provide the beginning date ('YYYY-MM-DD' format)
since_date = '2018-01-01 12:00:00'

# provide the end date
until_date = '2023-01-01 12:00:00'

### Run the scrapper function to extract the historical dataset and save

In [ ]:
for i in range (len(username_list)):
  # run the scrapper function
  feed_df = data_acq_search (str(username_list[i]), N, since_date, until_date)

  # convert the output into a dataframe
  df = pd.DataFrame (feed_df)

  # print the head of the dataframe
  print (df.head())

  # save the dataframe as csv to your Google Drive -- 
  #================= CHANGE THE PATH BELOW ACCORDING TO THE SEARCH ABOVE==================

  #df.to_csv ('/content/drive/My Drive/Twitter-Accounts-Historical-Data/Larrouturou/Bot/%s.csv' %username_list[i])
  #df.to_csv ('/content/drive/My Drive/Twitter-Accounts-Historical-Data/Larrouturou/Real/%s.csv' %username_list[i])

  #df.to_csv ('/content/drive/My Drive/Twitter-Accounts-Historical-Data/Alviinaalametsa/Bot/%s.csv' %username_list[i])
  df.to_csv ('/content/drive/My Drive/Twitter-Accounts-Historical-Data/Alviinaalametsa/Real/%s.csv' %username_list[i])

:: Acquiring Data for khab33b ::
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Empty DataFrame
Columns: []
Index: []
:: Acquiring Data for Hilale_pakistan ::
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Empty DataFrame
Columns: []
Index: []
:: Acquiring Data for Arjun_S_R ::
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
                    id      conversation_id    created_at  \
0  1609245124894261249  1609236169438494722  1.672509e+12   
1  1609242920007389190  1609009935144349696  1.672508e+12   
2  1609029221896290310  1608056944501178368  1.672457e+12   
3  1609026440095469569  1608735970131849217  1.672457e+12   
4  1609025699167490049  1608788083448901634  1.672457e+12   

                  date timezone place  \
0  2022-12-31 17:48:30    +0000         
1  2022-12-31 17:39:45    +0000         
2  2022-12-31 03:30:35    +0000         
3  2022-12-31 03:19:32    +0000         
